In [ ]:
import sys

sys.path.append("../../")

from syntemp.SynITS.its_extraction import ITSExtraction
from syntemp.SynUtils.utils import load_database, save_database

toy_sample = [
    {
        "balanced": True,
        "R-id": "USPTO_50K_31",
        "reactions": "C=C1C(=C)C2OC1C(=C)C2=C.C=CC(C)=O>>C=C1C(=C)C2OC1C1=C2CC(C(C)=O)CC1",
        "local_mapper": "[CH2:4]=[C:3]1[C:2](=[CH2:1])[CH:7]2[O:6][CH:5]1[C:9](=[CH2:10])[C:8]2=[CH2:16].[CH2:15]=[CH:11][C:12]([CH3:13])=[O:14]>>[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[O:6][CH:7]1[C:8]1=[C:9]2[CH2:10][CH:11]([C:12]([CH3:13])=[O:14])[CH2:15][CH2:16]1",
        "rxn_mapper": "[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[O:6][CH:7]1[C:8](=[CH2:16])[C:9]2=[CH2:15].[CH2:10]=[CH:11][C:12]([CH3:13])=[O:14]>>[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[O:6][CH:7]1[C:8]1=[C:9]2[CH2:10][CH:11]([C:12]([CH3:13])=[O:14])[CH2:15][CH2:16]1",
        "graphormer": "[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[C:10](=[CH2:11])[C:8](=[CH2:9])[CH:7]1[O:6]2.[CH3:15][C:14](=[O:16])[CH:13]=[CH2:12]>>[CH3:12][C:15]([CH:14]1[CH2:13][CH2:11][C:10]2=[C:8]([CH:7]3[O:6][CH:5]2[C:3](=[CH2:4])[C:2]3=[CH2:1])[CH2:9]1)=[O:16]",
        "Valid": True,
    }
]

# **1. ITS and Templates Extraction**


In [ ]:
from syntemp.SynVis.chemical_reaction_visualizer import ChemicalReactionVisualizer

reaction_vis = ChemicalReactionVisualizer()
vis_reaction = reaction_vis.visualize_reaction(
    toy_sample[0]["local_mapper"],
    show_atom_map=True,
    padding=0.01,
    img_size=(600, 100),
    bond_line_width=1,
)
vis_reaction

In [ ]:
import tempfile
import os
from IPython.display import SVG


def save_svg_to_file(svg_object):
    svg_data = svg_object.data
    with tempfile.NamedTemporaryFile(delete=False, suffix=".svg") as tmpfile:
        tmpfile.write(svg_data.encode("utf-8"))
        return tmpfile.name


from reportlab.graphics import renderPDF
from svglib.svglib import svg2rlg


def svg_file_to_pdf(svg_file, pdf_file):
    drawing = svg2rlg(svg_file)
    renderPDF.drawToFile(drawing, pdf_file)


svg_file_path = save_svg_to_file(vis_reaction)
svg_file_to_pdf(svg_file_path, f"./fig/vis_reaction.pdf")

from pdf2image import convert_from_path


def pdf_to_images(pdf_path, dpi=900):
    """
    Converts PDFs to images with an option to specify the DPI for higher quality.

    Args:
    pdf_paths (list): List of paths to PDF files.
    dpi (int): Dots per inch for conversion, higher values mean better quality.

    Returns:
    list: A list of PIL images of the first page of each PDF.
    """
    images = convert_from_path(pdf_path, dpi=dpi, first_page=1, last_page=1)[0]
    return images


images = pdf_to_images("./fig/vis_reaction.pdf")

In [ ]:
from syntemp.SynVis.its_visualizer import ITSVisualizer
from IPython.display import Image

its_vis = ITSVisualizer(toy_sample[0]["rxn_mapper"])
_its = Image(its_vis.draw_product_with_modified_bonds(showAtomMaps=True))
_rc = Image(its_vis.draw_highlighted_subgraph(showAtomMaps=True))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
import io
import numpy as np
from PIL import Image as PILImage


def display_images_in_subplot(
    image1, image2, image3, titles, save_path=None, show_grid=True
):
    """
    Display images in a customized subplot layout, with optional grid lines.

    Args:
        image1 (Image or PILImage): The first image to display in the large top subplot.
        image2 (Image or PILImage): The second image to display in the bottom left subplot.
        image3 (Image or PILImage): The third image to display in the bottom right subplot.
        titles (list of str): Titles for each subplot. Expected to have 3 entries.
        save_path (str, optional): Path to save the figure. If None, the figure is not saved.
        show_grid (bool, optional): Whether to show grid lines on the images.
    """
    # Set the aesthetic style of the plots and increase the title font size
    sns.set(style="whitegrid", rc={"axes.titlesize": 20, "axes.titleweight": "medium"})

    # Create a figure with a 2x2 grid
    fig, axs = plt.subplots(2, 2, figsize=(16, 9))

    # Remove the unused axes and use subplot to span the first row
    axs[0, 0].remove()  # Remove the first cell in the first row
    axs[0, 1].remove()  # Remove the second cell in the first row
    ax_big = fig.add_subplot(
        2, 2, (1, 2)
    )  # Add a big subplot that spans across the first row

    # Display the first image in the big subplot
    if isinstance(image1, Image):  # Check if the image is an IPython Image object
        image_stream = io.BytesIO(image1.data)
        pil_image = PILImage.open(image_stream)
        image_array = np.asarray(pil_image)
    else:
        image_array = np.asarray(image1)
    ax_big.imshow(image_array)
    ax_big.axis("off")
    ax_big.set_title(titles[0], fontsize=28, color="black")
    if show_grid:
        ax_big.grid(
            True, color="white", linestyle="--", linewidth=0.5, alpha=0.7
        )  # Apply visible grid lines

    # Display the second and third images in the remaining cells
    for i, img in enumerate([image2, image3]):
        if isinstance(img, Image):
            image_stream = io.BytesIO(img.data)
            pil_image = PILImage.open(image_stream)
            image_array = np.asarray(pil_image)
        else:
            image_array = np.asarray(img)
        axs[1, i].imshow(image_array)
        axs[1, i].axis("off")
        axs[1, i].set_title(titles[i + 1], fontsize=28, color="black")
        if show_grid:
            axs[1, i].grid(
                True, color="white", linestyle="--", linewidth=0.5, alpha=0.7
            )  # Apply visible grid lines

    plt.tight_layout(pad=1.0)
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches="tight")
    plt.show()


titles = ["A", "B", "C"]
display_images_in_subplot(
    images, _its, _rc, titles, save_path="./fig/Fig1_old_aam_its_rc.pdf"
)

In [ ]:
vis_reaction_1 = reaction_vis.visualize_reaction(
    toy_sample[0]["rxn_mapper"],
    show_atom_map=True,
    padding=0.01,
    img_size=(600, 100),
    bond_line_width=1,
)
vis_reaction_2 = reaction_vis.visualize_reaction(
    toy_sample[0]["graphormer"],
    show_atom_map=True,
    padding=0.01,
    img_size=(600, 100),
    bond_line_width=1,
)
vis_reaction_3 = reaction_vis.visualize_reaction(
    toy_sample[0]["local_mapper"],
    show_atom_map=True,
    padding=0.01,
    img_size=(600, 100),
    bond_line_width=1,
)

In [ ]:
import tempfile
import os
from IPython.display import SVG


def save_svg_to_file(svg_object):
    # Assuming svg_object.data holds the SVG string data
    svg_data = svg_object.data  # Adjust this based on your actual object structure
    # Create a temporary file to hold the SVG
    with tempfile.NamedTemporaryFile(delete=False, suffix=".svg") as tmpfile:
        tmpfile.write(svg_data.encode("utf-8"))
        return tmpfile.name


from reportlab.graphics import renderPDF
from svglib.svglib import svg2rlg


def svg_file_to_pdf(svg_file, pdf_file):
    drawing = svg2rlg(svg_file)
    renderPDF.drawToFile(drawing, pdf_file)


svg_file_path = save_svg_to_file(vis_reaction_1)
for key, value in enumerate([vis_reaction_1, vis_reaction_2, vis_reaction_3]):
    svg_file_path = save_svg_to_file(value)
    svg_file_to_pdf(svg_file_path, f"vis_{key}.pdf")

from pdf2image import convert_from_path


def pdf_to_images(pdf_paths, dpi=900):
    """
    Converts PDFs to images with an option to specify the DPI for higher quality.

    Args:
    pdf_paths (list): List of paths to PDF files.
    dpi (int): Dots per inch for conversion, higher values mean better quality.

    Returns:
    list: A list of PIL images of the first page of each PDF.
    """
    images = [
        convert_from_path(pdf_path, dpi=dpi, first_page=1, last_page=1)[0]
        for pdf_path in pdf_paths
    ]
    return images


# Example usage with file paths
pdf_paths = ["vis_0.pdf", "vis_1.pdf", "vis_2.pdf"]
images = pdf_to_images(pdf_paths)

import matplotlib.pyplot as plt


def display_images_in_subplot(images, titles, output_filename):
    """
    Displays a list of PIL images in a 1x3 subplot with high quality and reduced whitespace.

    Args:
    images (list): List of PIL.PpmImagePlugin.PpmImageFile objects.
    titles (list): List of titles for each subplot.
    output_filename (str): Filename for saving the output PDF.
    """
    # Set the size of the figure and the DPI for high resolution
    fig, axs = plt.subplots(
        3, 1, figsize=(18, 12), dpi=600
    )  # Adjust size and DPI as needed for balance between quality and file size

    # Display each image in a subplot with adjusted padding and aspect ratio
    for i, img in enumerate(images):
        axs[i].imshow(img)
        axs[i].set_title(
            titles[i], fontsize=24, color="navy"
        )  # Set the title for each subplot with enhanced styling
        axs[i].axis("off")  # Hide axes

    # Adjust layout to minimize white space
    plt.subplots_adjust(
        left=0.01, right=0.99, top=0.95, bottom=0.1, wspace=0.05, hspace=0.1
    )
    plt.tight_layout()

    # Save the figure to a PDF file with high quality
    fig.savefig(
        output_filename, format="pdf", dpi=600
    )  # Adjust DPI here if different from subplot creation
    plt.show()


images = [images[0], images[1], images[2]]
titles = ["A. RXNMapper", "B. Graphormer", "C. LocalMapper"]
display_images_in_subplot(images, titles, "./fig/Fig3a_old_aam_benchmark.pdf")

In [ ]:
from syntemp.SynITS.its_extraction import ITSExtraction

# Extract rules and graph rules based on Imaginary Transition State (ITS)
mapper_name = ["rxn_mapper", "graphormer", "local_mapper"]
its_graph, its_graph_wrong = ITSExtraction.parallel_process_smiles(
    toy_sample,
    mapper_name,
    n_jobs=2,
    verbose=1,
    export_full=False,
    check_method="RC",
)

In [ ]:
from syntemp.SynRule.rules_extraction import RuleExtraction

extract = RuleExtraction()
rules_tuple = extract.extract_reaction_rules(*its_graph_wrong[0]["local_mapper"])

from syntemp.SynVis.chemical_graph_visualizer import ChemicalGraphVisualizer

vis = ChemicalGraphVisualizer(seed=41)
vis.visualize_all(
    its_graph_wrong[0]["local_mapper"],
    rules_tuple,
    save_path="./fig/ITS_Rules_Graph.pdf",
    titles_row1=["A. Reactant graph", "B. ITS graph", "C.Product graph"],
    titles_row2=["D. Left graph", "E. Reaction center", "F. Right graph"],
    title_weight="medium",
    title_fontsize=20,
    figsize=(16, 9),
)

# **2. Issue of AAM**

In [ ]:
toy_sample = [
    {
        "R-id": "USPTO_50K_31",
        "local_mapper": "[CH2:4]=[C:3]1[C:2](=[CH2:1])[CH:7]2[O:6][CH:5]1[C:9](=[CH2:10])[C:8]2=[CH2:16].[CH2:15]=[CH:11][C:12]([CH3:13])=[O:14]>>[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[O:6][CH:7]1[C:8]1=[C:9]2[CH2:10][CH:11]([C:12]([CH3:13])=[O:14])[CH2:15][CH2:16]1",
        "rxn_mapper": "[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[O:6][CH:7]1[C:8](=[CH2:16])[C:9]2=[CH2:15].[CH2:10]=[CH:11][C:12]([CH3:13])=[O:14]>>[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[O:6][CH:7]1[C:8]1=[C:9]2[CH2:10][CH:11]([C:12]([CH3:13])=[O:14])[CH2:15][CH2:16]1",
        "graphormer": "[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[C:10](=[CH2:11])[C:8](=[CH2:9])[CH:7]1[O:6]2.[CH3:15][C:14](=[O:16])[CH:13]=[CH2:12]>>[CH3:12][C:15]([CH:14]1[CH2:13][CH2:11][C:10]2=[C:8]([CH:7]3[O:6][CH:5]2[C:3](=[CH2:4])[C:2]3=[CH2:1])[CH2:9]1)=[O:16]",
    }
]

In [ ]:
from syntemp.SynVis.chemical_reaction_visualizer import ChemicalReactionVisualizer

reaction_vis = ChemicalReactionVisualizer()
vis_reaction_1 = reaction_vis.visualize_reaction(
    toy_sample[0]["rxn_mapper"], show_atom_map=True
)
vis_reaction_2 = reaction_vis.visualize_reaction(
    toy_sample[0]["graphormer"], show_atom_map=True
)
vis_reaction_3 = reaction_vis.visualize_reaction(
    toy_sample[0]["local_mapper"], show_atom_map=True
)

In [ ]:
its_vis = ITSVisualizer(toy_sample[0]["rxn_mapper"])
img1 = Image(its_vis.draw_product_with_modified_bonds(showAtomMaps=False))

In [ ]:
its_vis = ITSVisualizer(toy_sample[0]["graphormer"])
img2 = Image(its_vis.draw_product_with_modified_bonds(showAtomMaps=False))

In [ ]:
its_vis = ITSVisualizer(toy_sample[0]["local_mapper"])
img3 = Image(its_vis.draw_product_with_modified_bonds(showAtomMaps=False))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Image
import io
import numpy as np
from PIL import Image as PILImage


def display_images_in_subplot(image1, image2, image3, titles, save_path=None):
    # Set the aesthetic style of the plots and increase the title font size
    sns.set(style="whitegrid", rc={"axes.titlesize": 18, "axes.titleweight": "bold"})

    # Create a figure with 1x3 subplot layout
    fig, axs = plt.subplots(1, 3, figsize=(18, 12))

    # Display each image in a subplot
    for i, img in enumerate([image1, image2, image3]):
        # Since IPython Image object stores data as bytes, we convert it for display
        image_stream = io.BytesIO(img.data)
        pil_image = PILImage.open(image_stream)
        axs[i].imshow(np.asarray(pil_image))
        axs[i].axis("off")  # Hide axes
        axs[i].set_title(
            titles[i], fontsize=16, color="navy"
        )  # Set the title for each subplot with enhanced styling

        # Optional: Uncomment if a grid is desired over the images
        axs[i].grid(True, which="both", color="gray", linestyle="-", linewidth=0.5)

    plt.tight_layout(pad=1, h_pad=0, w_pad=0)  # Adjust layout to make room for titles
    plt.subplots_adjust(
        top=1.6
    )  # Adjust the top margin to allow better space for the suptitle
    plt.suptitle("D. ITS Graph", fontsize=24, color="navy", weight="bold")

    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches="tight")
    plt.show()


display_images_in_subplot(
    img1,
    img2,
    img3,
    ["RXNMapper", "Graphormer", "LocalMapper"],
    save_path="./fig/Fig3b_old_aam_benchmark_its.pdf",
)